In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import timedelta


# Load Data

In [ ]:
# Optimized portfolio weights
weights = pd.read_csv("data/processed/Optimal_Portfolio.csv", index_col=0)["Weight"]

# Historical price data
tsla = pd.read_csv("data/processed/TSLA_processed.csv", index_col=0, parse_dates=True)
bnd = pd.read_csv("data/processed/BND_processed.csv", index_col=0, parse_dates=True)
spy = pd.read_csv("data/processed/SPY_processed.csv", index_col=0, parse_dates=True)

# Merge historical prices into one DataFrame
prices = pd.DataFrame({
    "TSLA": tsla["Adj Close"],
    "BND": bnd["Adj Close"],
    "SPY": spy["Adj Close"]
})

# Backtesting period (last year)
backtest_start = "2025-01-01"
backtest_end = "2026-01-15"
prices_bt = prices.loc[backtest_start:backtest_end]
prices_bt.head()


#  Compute Daily Returns

In [ ]:
returns_bt = prices_bt.pct_change().dropna()
returns_bt.head()


# Compute Strategy Portfolio Daily Returns

In [ ]:
strategy_returns = returns_bt.dot(weights)
strategy_cum = (1 + strategy_returns).cumprod()

# Benchmark 60% SPY / 40% BND
benchmark_weights = np.array([0.0, 0.4, 0.6])  # TSLA=0%, BND=40%, SPY=60%
benchmark_returns = returns_bt.dot(benchmark_weights)
benchmark_cum = (1 + benchmark_returns).cumprod()


# Plot Cumulative Returns

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(strategy_cum.index, strategy_cum, label="Strategy Portfolio")
plt.plot(benchmark_cum.index, benchmark_cum, label="Benchmark 60/40")
plt.title("Cumulative Returns Comparison")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.show()


# Performance Metrics

In [ ]:
def performance_metrics(returns):
    total_return = (1 + returns).prod() - 1
    annualized_return = (1 + total_return) ** (252 / len(returns)) - 1
    sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252)
    cum_returns = (1 + returns).cumprod()
    drawdown = 1 - cum_returns / cum_returns.cummax()
    max_drawdown = drawdown.max()
    return total_return, annualized_return, sharpe_ratio, max_drawdown


strategy_metrics = performance_metrics(strategy_returns)
benchmark_metrics = performance_metrics(benchmark_returns)

metrics_df = pd.DataFrame({
    "Metric": ["Total Return", "Annualized Return", "Sharpe Ratio", "Max Drawdown"],
    "Strategy": strategy_metrics,
    "Benchmark": benchmark_metrics
})

metrics_df


# Save Metrics

In [ ]:
metrics_df.to_csv("data/processed/Backtest_Metrics.csv", index=False)
